# Домашнее задание к уроку 2. Бейзлайны и детерминированные алгоритмы item-item

In [1]:
!pip install implicit

In [23]:
import implicit

In [24]:
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [5]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [6]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [7]:
items_weights = data_train.groupby('item_id').agg(sales=('sales_value', 'sum')).reset_index()
items_weights['weights'] = items_weights[['sales']].apply(lambda x: x / np.sum(x))
items_weights

,item_id,sales,weights
0,25671,20.94,2.969296e-06
1,26081,0.99,1.403822e-07
2,26093,1.59,2.254623e-07
3,26190,1.54,2.183723e-07
4,26355,1.98,2.807644e-07
...,...,...,...
86860,17381856,0.00,0.000000e+00
86861,17382205,7.99,1.132983e-06
86862,17383227,4.49,6.366828e-07
86863,17827644,2.50,3.545005e-07


In [8]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
#     items = np.array(items_weights['item_id'])
#     recs = np.random.choice(items, size=n, replace=False, p=items_weights['weight'].tolist())
    
#     return recs.tolist()

    items = np.array(items_weights['item_id'])
    items_weights = np.array(items_weights['weights'])
    recs = np.random.choice(items, size=n, replace=False)
    
    
    return recs.tolist()

In [9]:
%%time

# your_code
# items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
# items_weights = items_weights[items_weights['sales_value']>1]
# items_weights['weight']= np.log(items_weights['sales_value'] + 0.00001)
# items_weights['weight'] = items_weights['weight'] / items_weights['weight'].sum()
# items_weights.drop('sales_value', axis=1, inplace=True)

# result['weighted_random_recommendation'] = result['user_id'].apply\
#                                     (lambda x: weighted_random_recommendation(items_weights))



result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

result['weighted_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head(2)

CPU times: user 3.3 s, sys: 1.1 s, total: 4.39 s
Wall time: 4.42 s


,user_id,actual,weighted_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1795553, 1045887, 12456275, 12456256, 434858]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[81997, 839418, 8353376, 877051, 6773008]"


In [10]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [11]:
for name_col in result.columns[1:]:
#     print(name_col)
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0008:weighted_recommendation


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [12]:
# your_code

In [13]:
print(data_train.item_id.unique().shape)

(86865,)


In [14]:
? ItemItemRecommender

Object ` ItemItemRecommender` not found.


In [15]:
# Заведем фиктивный item_id (если юзер НЕ покупал товары из топ-5000, то он "купил" такой товар)
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [16]:
# перенумеруем пользователей и товары
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [17]:
%%time
for i in range(2,11):
    model = ItemItemRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)

    name = 'itemitem_k_'+str(i)
    result[name] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])

NameError: name 'ItemItemRecommender' is not defined

In [18]:
%%time
for i in range(2,11):
    model = CosineRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)

    name = 'cosine_k_'+str(i)
    result[name] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])

NameError: name 'CosineRecommender' is not defined

In [19]:
%%time
for i in range(2,11):
    model = CosineRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)

    name = 'cosine_k_'+str(i)
    result[name] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])

NameError: name 'CosineRecommender' is not defined

In [20]:
%%time
for i in range(2,11):
    model = TFIDFRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)

    name = 'tfidf_k_'+str(i)
    result[name] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])

NameError: name 'TFIDFRecommender' is not defined

In [21]:
result.shape,sparse_user_item.shape

((2042, 3), (2499, 86865))

In [22]:
result.head(2)

,user_id,actual,weighted_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1795553, 1045887, 12456275, 12456256, 434858]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[81997, 839418, 8353376, 877051, 6773008]"
